In [1]:
import pandas as pd

from pandas import DataFrame as df
import numpy as np
import csv
import pickle


In [2]:
otu = pd.read_csv('../data/AG_new/allbodysites/otu_table.txt', sep = '\t', index_col = 0)
print(otu.shape)

FileNotFoundError: File b'../data/AG_new/otu_table.txt' does not exist

In [31]:
otu.head()

,10317.000034640,10317.000063422,10317.000063446,10317.000005636,10317.000005637,10317.000005634,10317.000007117,10317.000005221,10317.000005598,10317.000007346,...,10317.000006824,10317.BLANK199.4H.r43,10317.BLANK107.r9,10317.000018049,10317.BLANK199.4D.r43,10317.BLANK127.3D.r28,10317.000013550,10317.BLANK.7.F.r9,10317.Blank33.F12.r6,10317.BLANK.AG80.12G.r20
OTUID,,,,,,,,,,,,,,,,,,,,,
4479946,5.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
988534,0.0,0.0,0.0,3.0,1.0,18.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4479944,0.0,0.0,6.0,0.0,0.0,0.0,0.0,1.0,13.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
370251,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
741867,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
#Throw samples with less than 5000 reads
sample_sums = np.sum(otu, axis = 0)

In [4]:
otu_filter = otu.loc[:, sample_sums > 5000]
print(otu_filter.shape)

(35511, 15475)


In [5]:
taxa_sums = np.sum(otu_filter, axis = 1)
otu_filter = otu_filter.loc[taxa_sums > 0, :]
print(otu_filter.shape)

(35275, 15475)


In [6]:
#Read in map file so we can decide on samples to include
mapping = pd.read_csv("data/AG_new/AG_mapping.txt", sep = "\t", index_col=0)
map_clean = mapping.loc[otu_filter.columns.values] #Keep samples if present in otu
map_clean = map_clean.reindex(sorted(map_clean.index.values))
otu_clean = otu_filter.reindex(sorted(otu_filter.columns.values), axis = 1)

C:\Users\ctata\Miniconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0,81) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\ctata\Miniconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  This is separate from the ipykernel package so we can avoid doing imports until


# Decide on what samples should be included in the embedding calculations, and name files accordingly

In [36]:
#Select all body sites

#otu_table_name = "data/AG_new/otu_filtered_AG_02perc_allbodysites.csv"
#otu_train_table_name = "data/AG_new/otu_filtered_train_AG_02perc_allbodysites.csv"
#otu_test_table_name = "data/AG_new/otu_filtered_test_AG_02perc_allbodysites.csv"
#test_sample_file = "data/AG_new/AG_test_samples_allbodysites.obj"

In [7]:
#Select only feces samples
otu_use = otu_clean.loc[:, map_clean["BODY_SITE"] == "UBERON:feces"]
map_use = map_clean.loc[map_clean["BODY_SITE"] == "UBERON:feces", :]

otu_table_name = "data/AG_new/feces/otu_filtered_AG_07perc_feces.csv"
otu_train_table_name = "data/AG_new/feces/otu_filtered_train_AG_07perc_feces.csv"
otu_test_table_name = "data/AG_new/feces/otu_filtered_test_AG_07perc_feces.csv"
test_sample_file = "data/AG_new/feces/AG_test_samples_feces.obj"
glove_output_file = "data/AG_new/feces/glove_input_07perc_feces.txt"

In [8]:
thresh = 0.0007 * otu_use.shape[1] # at least 1 % of samples (10 in this case)
print("Only keep taxa present in greater than " + str(thresh) + " samples")
binary = otu_use > 0
keep = binary.sum(axis = 1) >= thresh# Should be ntaxa
binary = binary.loc[keep, :]
print("We will keep " + str(keep.sum()) +" taxa")


Only keep taxa present in greater than 6.3924 samples
We will keep 9162 taxa


In [39]:
#Split out a test set

np.random.seed(15)
test_samples = np.random.choice(binary.columns.values, 1500)
f = open(test_sample_file, "wb")
pickle.dump(test_samples, f)
f.close()

binary_train = binary.loc[:, [not(i in test_samples) for i in binary.columns.values ]]
otu_train = otu_use.loc[:, [not(i in test_samples) for i in binary.columns.values ]]
otu_test = otu_use[test_samples]


In [40]:
#Test text output with small file
#otu_filter["#OTU ID"].head()
#tmp = otu_filter.head()
#tmp = tmp.iloc[:, [0,1,2,3,4]]
#tmp.head()
#tmp.to_csv("silva/test_out.csv", sep = "\t", index = False)
print(otu_train.shape)
print(otu_test.shape)
print(otu_filter.shape)

(35275, 7744)
(35275, 1500)
(35275, 15475)


In [ ]:
#Check that taxa have their names

In [41]:
otu_train_filter = otu_train.loc[np.array(keep), :]
otu_test_filter = otu_test.loc[np.array(keep), :]
otu_filter = otu_filter.loc[keep, :]

otu_filter.to_csv(otu_table_name, sep = "\t", index = True)
otu_train_filter.to_csv(otu_train_table_name, sep = "\t", index = True)
otu_test_filter.to_csv(otu_test_table_name, sep = "\t", index = True)

In [49]:
taxa_by_taxa = np.dot(binary, binary.T)

In [43]:
file_text = []
for i in range(binary_train.shape[1]):
    taxa_share = binary_train.index[binary_train.iloc[:,i] == True]
    file_text.append(taxa_share)

In [44]:
lengths = []
for taxa_list in file_text:
    lengths.append(len(taxa_list))
max_val = max(lengths)
max_ind = np.argmax(lengths)
print("Max number of taxa in any given sample is: " + str(max_val))
print("Found at " + str(max_ind))


Max number of taxa in any given sample is: 1792
Found at 3949


In [47]:
#Write Glove input file
with open(glove_output_file, mode = 'w', newline='') as file:
    writer = csv.writer(file, delimiter = "\t", quoting = csv.QUOTE_NONE, escapechar = '')
    for taxa_list in file_text:
        writer.writerow(taxa_list)
file.close()
